In [1]:
import pandas as pd
import numpy as np
import re
import math
from cleantext import clean
import matplotlib.pyplot as plt
import nltk
from nltk.stem.snowball import SnowballStemmer
from collections import Counter
import csv

nltk.download('punkt')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 150)

C:\Users\astri\AppData\Local\Temp\ipykernel_26872\964166201.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\astri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
#det her er ikke færdigt
from sklearn.model_selection import train_test_split
#remove unknowns
relevant = ['fake','reliable']

news_data = csv.read("small_datasetprocessed.csv")

#sorting data
data = news_data[news_data["simple_type"].isin(relevant) == True] #fjerne ikke-relevante rækker, simple type tracker definitionen af fake news 

print("row count after removal:", len(data))
print(list(data))

X = data.drop(["type"],axis=1).drop(["simple_type"],axis=1) #ikke sikker på de skal være i en liste
#tjek lige om der er andre mærkelige kolonner vi har lavet i det?
y = data["simple_type"] #eller type?

X_train, X_val, y_train, y_val= train_test_split(X,y,test_size=0.2,random_state=0,shuffle=True)
X_val, X_test, y_val, y_test = train_test_split(X_val,y_val, test_size=0.5,random_state=0,shuffle=True)

print(len(X_train),len(X_val),len(X_test))
print("features", X_test.head())      
#tjek om de stadig passer sammen
#spørg til:
#1: I opgaven virker det som om at vi først skal splitte data før vi deler i fake og reliable - det giver ikke mening umiddelbart


In [ ]:
#baseline model 1: single-feature logistic regression, length of article
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

def bin_target(x):
    if x=="reliable":
        return 1
    if x=="fake":
        return 0

article_length_train = X_train["content_stemmed"].apply(lambda x: len(x)) #get feature with length of article (overvej om det er stemmed man skal gøre det på?)
article_length_test = X_test["content_stemmed"].apply(lambda x: len(x)) #get feature with length of article (overvej om det er stemmed man skal gøre det på?)

bin_y_train = y_train.apply(lambda x:bin_target(x)) #get binary target
bin_y_test = y_test.apply(lambda x:bin_target(x)) #get binary target

model = LogisticRegression()
reg = model.fit(pd.DataFrame(article_length_train),bin_y_train)

y_pred = model.predict(pd.DataFrame(article_length_test)) #er det test eller validation?
acc = accuracy_score(y_pred,bin_y_test)

#metadata kunne være hjemmeside?
print(acc)


In [ ]:
#baseline model 2: single-feature logistic regression, mean word length
bin_y_train = y_train.apply(lambda x:bin_target(x)) #get binary target
bin_y_test = y_test.apply(lambda x:bin_target(x)) #get binary target

model = LogisticRegression()
reg = model.fit(pd.DataFrame(X_train["mean_word_length"]),bin_y_train)

y_pred = model.predict(pd.DataFrame(X_test["mean_word_length"])) #er det test eller validation?
acc = accuracy_score(y_pred,bin_y_test)

#metadata kunne være hjemmeside?
print(acc)
